# End-to-End Data Cleaning Pipeline with Raha and Baran (Minimal and Sequential)
We build an end-to-end data cleaning pipeline with our configuration-free error detection and correction systems, Raha and Baran.

In [1]:
import pandas
import IPython.display

import raha

## Error Detection with Raha

### 1. Instantiating the Detection Class
We first instantiate the `Detection` class.

In [2]:
app_1 = raha.Detection()

# How many tuples would you label?
app_1.LABELING_BUDGET = 20

# Would you like to see the logs?
app_1.VERBOSE = True

### 2. Instantiating the Dataset
We next load and instantiate the dataset object.

In [3]:
dataset_dictionary = {
    "name": "flights",
    "path": "../datasets/flights/dirty.csv",
    "clean_path": "../datasets/flights/clean.csv"
}
d = app_1.initialize_dataset(dataset_dictionary)
d.dataframe.head()

,tuple_id,src,flight,sched_dep_time,act_dep_time,sched_arr_time,act_arr_time
0,1,aa,AA-3859-IAH-ORD,7:10 a.m.,7:16 a.m.,9:40 a.m.,9:32 a.m.
1,2,aa,AA-1733-ORD-PHX,7:45 p.m.,7:58 p.m.,10:30 p.m.,
2,3,aa,AA-1640-MIA-MCO,6:30 p.m.,,7:25 p.m.,
3,4,aa,AA-518-MIA-JFK,6:40 a.m.,6:54 a.m.,9:25 a.m.,9:28 a.m.
4,5,aa,AA-3756-ORD-SLC,12:15 p.m.,12:41 p.m.,2:45 p.m.,2:50 p.m.


In [4]:
d.dataframe

,tuple_id,src,flight,sched_dep_time,act_dep_time,sched_arr_time,act_arr_time
0,1,aa,AA-3859-IAH-ORD,7:10 a.m.,7:16 a.m.,9:40 a.m.,9:32 a.m.
1,2,aa,AA-1733-ORD-PHX,7:45 p.m.,7:58 p.m.,10:30 p.m.,
2,3,aa,AA-1640-MIA-MCO,6:30 p.m.,,7:25 p.m.,
3,4,aa,AA-518-MIA-JFK,6:40 a.m.,6:54 a.m.,9:25 a.m.,9:28 a.m.
4,5,aa,AA-3756-ORD-SLC,12:15 p.m.,12:41 p.m.,2:45 p.m.,2:50 p.m.
...,...,...,...,...,...,...,...
2371,2372,world-flight-tracker,UA-3099-PHX-PHL,11:55 a.m.,11:43 a.m.,6:17 p.m.,5:38 p.m.
2372,2373,world-flight-tracker,AA-4198-ORD-CLE,10:40 a.m.,10:54 a.m.,12:55 p.m.,12:50 p.m.
2373,2374,world-flight-tracker,CO-45-EWR-MIA,4:00 p.m.,3:58 p.m.,7:05 p.m.,6:36 p.m.
2374,2375,world-flight-tracker,AA-3809-PHX-LAX,6:00 a.m.,6:10 a.m.,6:40 a.m.,6:19 a.m.


### 3. Running Error Detection Strategies
Raha runs (all or the promising) error detection strategies on the dataset. This step could take a while because all the strategies should be run on the dataset. 

In [7]:
app_1.run_strategies(d)

99 cells are detected by ["PVD", ["src", "z"]].498 cells are detected by ["PVD", ["flight", "P"]].

616 cells are detected by ["PVD", ["tuple_id", "4"]].
2 cells are detected by ["PVD", ["sched_arr_time", "P"]].871 cells are detected by ["PVD", ["flight", "4"]].

348 cells are detected by ["PVD", ["act_arr_time", "7"]].
85 cells are detected by ["PVD", ["sched_dep_time", "e"]].
745 cells are detected by ["PVD", ["src", "h"]].
2 cells are detected by ["PVD", ["sched_dep_time", "n"]].
436 cells are detected by ["PVD", ["src", "c"]].
591 cells are detected by ["PVD", ["act_arr_time", "4"]].
131 cells are detected by ["PVD", ["sched_dep_time", "9"]].
36 cells are detected by ["PVD", ["act_dep_time", "d"]].
2329 cells are detected by ["PVD", ["flight", "A"]].
35 cells are detected by ["PVD", ["act_dep_time", ")"]].196 cells are detected by ["PVD", ["sched_dep_time", "6"]].

741 cells are detected by ["PVD", ["sched_arr_time", "0"]].
1503 cells are detected by ["PVD", ["sched_dep_time", "m"]

### 4. Generating Features
Raha then generates a feature vector for each data cell based on the output of error detection strategies. 

In [8]:
app_1.generate_features(d)

40 Features are generated for column 0.
65 Features are generated for column 1.
62 Features are generated for column 2.
65 Features are generated for column 3.
71 Features are generated for column 4.
65 Features are generated for column 5.
86 Features are generated for column 6.


### 5. Building Clusters
Raha next builds a hierarchical clustering model for our clustering-based sampling approach.

In [9]:
app_1.build_clusters(d)

A hierarchical clustering model is built for column 0.
A hierarchical clustering model is built for column 1.
A hierarchical clustering model is built for column 2.
A hierarchical clustering model is built for column 3.
A hierarchical clustering model is built for column 4.
A hierarchical clustering model is built for column 5.
A hierarchical clustering model is built for column 6.


### 6. Interactive Tuple Sampling and Labeling
Raha then iteratively samples a tuple. We should label data cells of each sampled tuple.

In [10]:
while len(d.labeled_tuples) < app_1.LABELING_BUDGET:
    app_1.sample_tuple(d)
    if d.has_ground_truth:
        app_1.label_with_ground_truth(d)
    else:
        print("Label the dirty cells in the following sampled tuple.")
        sampled_tuple = pandas.DataFrame(data=[d.dataframe.iloc[d.sampled_tuple, :]], columns=d.dataframe.columns)
        IPython.display.display(sampled_tuple)
        for j in range(d.dataframe.shape[1]):
            cell = (d.sampled_tuple, j)
            value = d.dataframe.iloc[cell]
            correction = input("What is the correction for value '{}'? Type in the same value if it is not erronous.\n".format(value))
            user_label = 1 if value != correction else 0
            d.labeled_cells[cell] = [user_label, correction]
        d.labeled_tuples[d.sampled_tuple] = 1

Tuple 2250 is sampled.
Tuple 2250 is labeled.
Tuple 1010 is sampled.
Tuple 1010 is labeled.
Tuple 712 is sampled.
Tuple 712 is labeled.
Tuple 225 is sampled.
Tuple 225 is labeled.
Tuple 1424 is sampled.
Tuple 1424 is labeled.
Tuple 2175 is sampled.
Tuple 2175 is labeled.
Tuple 2343 is sampled.
Tuple 2343 is labeled.
Tuple 1325 is sampled.
Tuple 1325 is labeled.
Tuple 2076 is sampled.
Tuple 2076 is labeled.
Tuple 597 is sampled.
Tuple 597 is labeled.
Tuple 1801 is sampled.
Tuple 1801 is labeled.
Tuple 1058 is sampled.
Tuple 1058 is labeled.
Tuple 1991 is sampled.
Tuple 1991 is labeled.
Tuple 1381 is sampled.
Tuple 1381 is labeled.
Tuple 1467 is sampled.
Tuple 1467 is labeled.
Tuple 795 is sampled.
Tuple 795 is labeled.
Tuple 518 is sampled.
Tuple 518 is labeled.
Tuple 1502 is sampled.
Tuple 1502 is labeled.
Tuple 899 is sampled.
Tuple 899 is labeled.
Tuple 244 is sampled.
Tuple 244 is labeled.


Tuple 1234 is sampled.
Tuple 1234 is labeled.
Tuple 1385 is sampled.
Tuple 1385 is labeled.
Tuple 179 is sampled.
Tuple 179 is labeled.
Tuple 816 is sampled.
Tuple 816 is labeled.
Tuple 2358 is sampled.
Tuple 2358 is labeled.
Tuple 476 is sampled.
Tuple 476 is labeled.
Tuple 1415 is sampled.
Tuple 1415 is labeled.
Tuple 989 is sampled.
Tuple 989 is labeled.
Tuple 610 is sampled.
Tuple 610 is labeled.
Tuple 1304 is sampled.
Tuple 1304 is labeled.
Tuple 1973 is sampled.
Tuple 1973 is labeled.
Tuple 1874 is sampled.
Tuple 1874 is labeled.
Tuple 1126 is sampled.
Tuple 1126 is labeled.
Tuple 1789 is sampled.
Tuple 1789 is labeled.
Tuple 1672 is sampled.
Tuple 1672 is labeled.
Tuple 796 is sampled.
Tuple 796 is labeled.
Tuple 821 is sampled.
Tuple 821 is labeled.
Tuple 2192 is sampled.
Tuple 2192 is labeled.
Tuple 294 is sampled.
Tuple 294 is labeled.
Tuple 1965 is sampled.
Tuple 1965 is labeled.

### 7. Propagating User Labels
Raha then propagates each user label through its cluster.

Raha then propagates each user label through its cluster.

In [11]:
app_1.propagate_labels(d)

The number of labeled data cells increased from 140 to 12426.


In [13]:
d.dataframe

,tuple_id,src,flight,sched_dep_time,act_dep_time,sched_arr_time,act_arr_time
0,1,aa,AA-3859-IAH-ORD,7:10 a.m.,7:16 a.m.,9:40 a.m.,9:32 a.m.
1,2,aa,AA-1733-ORD-PHX,7:45 p.m.,7:58 p.m.,10:30 p.m.,
2,3,aa,AA-1640-MIA-MCO,6:30 p.m.,,7:25 p.m.,
3,4,aa,AA-518-MIA-JFK,6:40 a.m.,6:54 a.m.,9:25 a.m.,9:28 a.m.
4,5,aa,AA-3756-ORD-SLC,12:15 p.m.,12:41 p.m.,2:45 p.m.,2:50 p.m.
...,...,...,...,...,...,...,...
2371,2372,world-flight-tracker,UA-3099-PHX-PHL,11:55 a.m.,11:43 a.m.,6:17 p.m.,5:38 p.m.
2372,2373,world-flight-tracker,AA-4198-ORD-CLE,10:40 a.m.,10:54 a.m.,12:55 p.m.,12:50 p.m.
2373,2374,world-flight-tracker,CO-45-EWR-MIA,4:00 p.m.,3:58 p.m.,7:05 p.m.,6:36 p.m.
2374,2375,world-flight-tracker,AA-3809-PHX-LAX,6:00 a.m.,6:10 a.m.,6:40 a.m.,6:19 a.m.


### 8. Predicting Labels of Data Cells
Raha then trains and applies one classifier per data column to predict the label of the rest of data cells.

In [15]:
app_1.predict_labels(d)

A classifier is trained and applied on column 0.
A classifier is trained and applied on column 1.
A classifier is trained and applied on column 2.
A classifier is trained and applied on column 3.
A classifier is trained and applied on column 4.
A classifier is trained and applied on column 5.
A classifier is trained and applied on column 6.


### 9. Storing Results
Raha can also store the error detection results.

In [16]:
app_1.store_results(d)

The results are stored in ../datasets/flights/raha-baran-results-flights/error-detection/detection.dataset.


### 10. Evaluating the Error Detection Task
We can finally evaluate our error detection task.

In [17]:
p, r, f = d.get_data_cleaning_evaluation(d.detected_cells)[:3]
print("Raha's performance on {}:\nPrecision = {:.2f}\nRecall = {:.2f}\nF1 = {:.2f}".format(d.name, p, r, f))

Raha's performance on flights:
Precision = 0.92
Recall = 0.79
F1 = 0.85


# Error Correction with Baran

### 1. Instantiating the Correction Class
We first instantiate the `Correction` class.

In [13]:
app_2 = raha.Correction()

# How many tuples would you label?
app_2.LABELING_BUDGET = 20

# Would you like to see the logs?
app_2.VERBOSE = True

### 2. Initializing the Dataset Object
We next initialize the dataset object.

In [14]:
d = app_2.initialize_dataset(d)
d.dataframe.head()

,tuple_id,src,flight,sched_dep_time,act_dep_time,sched_arr_time,act_arr_time
0,1,aa,AA-3859-IAH-ORD,7:10 a.m.,7:16 a.m.,9:40 a.m.,9:32 a.m.
1,2,aa,AA-1733-ORD-PHX,7:45 p.m.,7:58 p.m.,10:30 p.m.,
2,3,aa,AA-1640-MIA-MCO,6:30 p.m.,,7:25 p.m.,
3,4,aa,AA-518-MIA-JFK,6:40 a.m.,6:54 a.m.,9:25 a.m.,9:28 a.m.
4,5,aa,AA-3756-ORD-SLC,12:15 p.m.,12:41 p.m.,2:45 p.m.,2:50 p.m.


### 3. Initializing the Error Corrector Models
Baran initializes the error corrector models.

In [15]:
app_2.initialize_models(d)

The error corrector models are initialized.


### 4. Interactive Tuple Sampling, Labeling, Model updating, Feature Generation, and Correction Prediction
Baran then iteratively samples a tuple. We should label data cells of each sampled tuple. It then udpates the models accordingly and generates a feature vector for each pair of a data error and a correction candidate. Finally, it trains and applies a classifier to each data column to predict the final correction of each data error. Since we already labeled tuples for Raha, we use the same labeled tuples and do not label new tuples here.

In [ ]:
# while len(d.labeled_tuples) < app_2.LABELING_BUDGET:
#     app_2.sample_tuple(d)
#     if d.has_ground_truth:
#         app_2.label_with_ground_truth(d)
#     else:
#         print("Label the dirty cells in the following sampled tuple.")
#         sampled_tuple = pandas.DataFrame(data=[d.dataframe.iloc[d.sampled_tuple, :]], columns=d.dataframe.columns)
#         IPython.display.display(sampled_tuple)
#         for j in range(d.dataframe.shape[1]):
#             cell = (d.sampled_tuple, j)
#             value = d.dataframe.iloc[cell]
#             correction = input("What is the correction for value '{}'? Type in the same value if it is not erronous.\n".format(value))
#             user_label = 1 if value != correction else 0
#             d.labeled_cells[cell] = [user_label, correction]
#         d.labeled_tuples[d.sampled_tuple] = 1
#     app_2.update_models(d)
#     app_2.predict_corrections(d)

for si in d.labeled_tuples:
    d.sampled_tuple = si
    app_2.update_models(d)
    app_2.predict_corrections(d)

### 5. Storing Results
Baran can also store the error correction results.

In [17]:
app_2.store_results(d)

The results are stored in ../datasets/flights/raha-baran-results-flights/error-correction/correction.dataset.


### 6. Evaluating the Error Correction Task
We can finally evaluate our error correction task.

In [18]:
p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
print("Baran's performance on {}:\nPrecision = {:.2f}\nRecall = {:.2f}\nF1 = {:.2f}".format(d.name, p, r, f))

Baran's performance on flights:
Precision = 0.88
Recall = 0.53
F1 = 0.66
